# QLoRA Tuning on Llama2

We will use the QLoRA technique to fine-tune the model in 4-bit precision and optimize VRAM usage.

Techniques applying:
- Quantization: HuggingFace Transformers has integrated optimum API to perform GPTQ quantization on LM. We can load and quantize our models in 8,4,3 or even 2 bits without a big drop of performance and still achieve faster inference speeds. This is achieved with the `BitsAndBytesConfig`. 
- LoRA: Stands for Low-rank Adaptation. It's widely used and effective for training custom LLMs. Read the paper [here](https://arxiv.org/abs/2305.14314).
- When you put quantization and LoRA together, we get QLoRA. Which, theoretically, reduces memory usage well.

In [1]:
import os
import pandas as pd
import creds

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer


/home/bobby/code-repo/practices/llm-examples/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Prepare some variables
## model from HF hub
base_model_name = 'meta-llama/Llama-2-7b-hf'

## New Insturctional Dataset
instructional_dataset = 'datasets/stylometry/zhongxing0129-authorlist_train-v2.csv'

## Folder name to store finetuned model
folder_name = 'meta-llama2-7b-stylometry'

### 1. Load the Dataset

In [6]:
dataset = pd.read_csv(instructional_dataset)

import datasets
dataset = datasets.Dataset.from_pandas(dataset)

In [7]:
dataset

Dataset({
    features: ['text_a', 'label_a', 'text_b', 'label_b', 'model_response', 'text'],
    num_rows: 2800
})

In [8]:
## view the dataset
print(pd.DataFrame(dataset).iloc[0]['text'])

<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> Author 0 wrote this: 'The observations of her uncle and aunt now began; and each of them
pronounced him to be infinitely superior to any thing they had expected.
"He is perfectly well behaved, polite, and unassuming," said her uncle.'. Did Author 0 also write this: 'The observations of her uncle and aunt now began; and each of them
pronounced him to be infinitely superior to any thing they had expected.
"He is perfectly well behaved, polite, and unassuming," sa

In [9]:
print(len(dataset))

## This is good. Optimally, we want the instructional prompts to be ~1000.

2800


### 2. Prepare 4-bit quantization configuration

In [2]:
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True, # data will be loaded in 4-bit format
    bnb_4bit_quant_type='nf4', # a quantizsation type
    bnb_4bit_compute_dtype=compute_dtype, # torch's float16
    bnb_4bit_use_double_quant=False # double quantization will not be used
)

### 3. Load Llama2 Model

In [93]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config, 
    device_map='auto', # automatically sets the device mapping
    token = creds.HUGGINGFACE_TOKEN
)

model.config.use_cache = False # disables the use of cache in the model config
model.config.pretraining_tp = 0 # sets the pretraining temperature parameter to 1 in the model config

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]


### 4. Load Llama2 Tokenizer

In [94]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name,trust_remote_code=True,token=creds.HUGGINGFACE_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### 5. Preparing the PEFT Parameters

In [95]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

### 6. Training Parameters

In [96]:
training_params = TrainingArguments(
    output_dir="./results/meta-llama2-7b-stylometry",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### 7. Model Fine-tuning

In [97]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text", # specifies the field in the dataset that contains text to be processed
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/bobby/code-repo/practices/llm-examples/env/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 2800/2800 [00:00<00:00, 8842.37 examples/s]


In [98]:
## train the model
trainer.train()

{'loss': 2.0469, 'grad_norm': 0.7105928659439087, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 1.052, 'grad_norm': 0.25334006547927856, 'learning_rate': 0.0002, 'epoch': 0.07}


KeyboardInterrupt: 

In [76]:
## train the model
trainer.train()

{'loss': 2.0469, 'grad_norm': 0.7105904817581177, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 1.052, 'grad_norm': 0.2532055377960205, 'learning_rate': 0.0002, 'epoch': 0.07}
{'loss': 1.1658, 'grad_norm': 0.3081775903701782, 'learning_rate': 0.0002, 'epoch': 0.11}
{'loss': 0.6258, 'grad_norm': 0.41304895281791687, 'learning_rate': 0.0002, 'epoch': 0.14}
{'loss': 0.9365, 'grad_norm': 0.47724291682243347, 'learning_rate': 0.0002, 'epoch': 0.18}
{'loss': 0.3466, 'grad_norm': 0.36514127254486084, 'learning_rate': 0.0002, 'epoch': 0.21}
{'loss': 0.6008, 'grad_norm': 0.553030252456665, 'learning_rate': 0.0002, 'epoch': 0.25}
{'loss': 0.1607, 'grad_norm': 0.22730310261249542, 'learning_rate': 0.0002, 'epoch': 0.29}
{'loss': 0.338, 'grad_norm': 0.3150671422481537, 'learning_rate': 0.0002, 'epoch': 0.32}
{'loss': 0.0817, 'grad_norm': 0.37636348605155945, 'learning_rate': 0.0002, 'epoch': 0.36}
{'loss': 0.2627, 'grad_norm': 0.5009521245956421, 'learning_rate': 0.0002, 'epoch': 0.39}
{'loss':

TrainOutput(global_step=700, training_loss=0.31451486332075934, metrics={'train_runtime': 2911.5729, 'train_samples_per_second': 0.962, 'train_steps_per_second': 0.24, 'train_loss': 0.31451486332075934, 'epoch': 1.0})

In [47]:
## train the model
trainer.train()

{'loss': 2.1403, 'grad_norm': 0.5620078444480896, 'learning_rate': 0.0002, 'epoch': 0.04}
{'loss': 1.8612, 'grad_norm': 0.8981637358665466, 'learning_rate': 0.0002, 'epoch': 0.07}
{'loss': 1.5978, 'grad_norm': 0.6241077184677124, 'learning_rate': 0.0002, 'epoch': 0.11}
{'loss': 1.0918, 'grad_norm': 0.9207419157028198, 'learning_rate': 0.0002, 'epoch': 0.14}
{'loss': 1.2188, 'grad_norm': 0.6704062223434448, 'learning_rate': 0.0002, 'epoch': 0.18}
{'loss': 0.5832, 'grad_norm': 1.207627534866333, 'learning_rate': 0.0002, 'epoch': 0.21}
{'loss': 0.7325, 'grad_norm': 0.664181649684906, 'learning_rate': 0.0002, 'epoch': 0.25}
{'loss': 0.2873, 'grad_norm': 2.9383227825164795, 'learning_rate': 0.0002, 'epoch': 0.29}
{'loss': 0.405, 'grad_norm': 0.8287302851676941, 'learning_rate': 0.0002, 'epoch': 0.32}
{'loss': 0.1596, 'grad_norm': 0.7628010511398315, 'learning_rate': 0.0002, 'epoch': 0.36}
{'loss': 0.3239, 'grad_norm': 0.8792100548744202, 'learning_rate': 0.0002, 'epoch': 0.39}
{'loss': 0.13

TrainOutput(global_step=700, training_loss=0.43931936502456664, metrics={'train_runtime': 2144.5585, 'train_samples_per_second': 1.306, 'train_steps_per_second': 0.326, 'train_loss': 0.43931936502456664, 'epoch': 1.0})

In [77]:
## save the model and tokenizer
trainer.model.save_pretrained(folder_name)#, token = creds.HUGGINGFACE_TOKEN)
trainer.tokenizer.save_pretrained(folder_name)

## be sure to login to the HF CLI first so that you can save the pretrained. ALternatively, you can use token = creds.HUGGINGFACE_TOKEN

('meta-llama2-7b-stylometry/tokenizer_config.json',
 'meta-llama2-7b-stylometry/special_tokens_map.json',
 'meta-llama2-7b-stylometry/tokenizer.json')

In [99]:
## load the model you had saved
loaded_model = AutoModelForCausalLM.from_pretrained(f"./{folder_name}",
                                                    quantization_config = quant_config,
                                                    device_map='auto')
loaded_model.config.use_cache = False
loaded_model.config.pretraining_tp = 0

loaded_tokenizer = AutoTokenizer.from_pretrained(f"./{folder_name}", trust_remote_code=True)
loaded_tokenizer.pad_token = loaded_tokenizer.eos_token
loaded_tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


### 8. Inferencing

In [100]:
inf_dataset = load_dataset('Zhongxing0129/authorlist_test', trust_remote_code=True, split = 'train')
inf_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 646
})

In [101]:
pd.DataFrame(inf_dataset).iloc[20]

text     The Cossacks sold the horse for two gold piece...
label                                                    2
Name: 20, dtype: object

In [102]:
pd.DataFrame(inf_dataset)

,text,label
0,"What with the novelty of this cookery, the exc...",3
1,"“Another year or two may do much towards it,” ...",0
2,“Everything. I can’t act except from the heart...,2
3,Konstantin Levin felt himself morally pinned a...,2
4,“Have the goodness to give me a little glass o...,3
...,...,...
641,"About the middle of the day, Mrs. Jennings wen...",0
642,Dorian Gray turned and looked at him. “I belie...,1
643,In a corner of the hut stood a standard captur...,2
644,Pestsov liked thrashing an argument out to the...,2


In [106]:
index = 2

author_a = dataset['label_a'][index]
text_a = dataset['text_a'][index]
text_b= pd.DataFrame(inf_dataset)['text'][index]

prompt = f"Author {author_a} wrote this: '{text_a}'. Did Author {author_a} also write this: '{text_b}'? Your response will either be yes, no or unsure."
prompt

'Author 0 wrote this: \'The observations of her uncle and aunt now began; and each of them\npronounced him to be infinitely superior to any thing they had expected.\n"He is perfectly well behaved, polite, and unassuming," said her uncle.\'. Did Author 0 also write this: \'“Everything. I can’t act except from the heart, and you act from\nprinciple. I liked you simply, but you most likely only wanted to save\nme, to improve me.”\'? Your response will either be yes, no or unsure.'

In [107]:
## Inference with SAVED MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

pipe_loaded = pipeline(task="text-generation", model=loaded_model, tokenizer=loaded_tokenizer, max_length=300)
result_loaded = pipe_loaded(f"<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> {prompt} [/INST]")
print(result_loaded[0]['generated_text'])

<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> Author 0 wrote this: 'The observations of her uncle and aunt now began; and each of them
pronounced him to be infinitely superior to any thing they had expected.
"He is perfectly well behaved, polite, and unassuming," said her uncle.'. Did Author 0 also write this: '“Everything. I can’t act except from the heart, and you act from
principle. I liked you simply, but you most likely only wanted to save
me, to improve me.”'? Your response will either be yes, no or 

In [105]:
## Inference with SAVED MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

pipe_loaded = pipeline(task="text-generation", model=loaded_model, tokenizer=loaded_tokenizer, max_length=300)
result_loaded = pipe_loaded(f"<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> {prompt} [/INST]")
print(result_loaded[0]['generated_text'])

<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. <</SYS>> Author 0 wrote this: 'The observations of her uncle and aunt now began; and each of them
pronounced him to be infinitely superior to any thing they had expected.
"He is perfectly well behaved, polite, and unassuming," said her uncle.'. Did Author 0 also write this: '“Another year or two may do much towards it,” he gravely replied; “but
however there is still a great deal to be done. There is not a stone
laid of Fanny’s green-house, and nothing but the plan of t

In [87]:
## Inference with SAVED MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

pipe_loaded = pipeline(task="text-generation", model=loaded_model, tokenizer=loaded_tokenizer, max_length=300)
result_loaded = pipe_loaded(f"<s>[INST] {prompt} [/INST]")
print(result_loaded[0]['generated_text'])

<s>[INST] Author 0 wrote this: 'The observations of her uncle and aunt now began; and each of them
pronounced him to be infinitely superior to any thing they had expected.
"He is perfectly well behaved, polite, and unassuming," said her uncle.'. Did Author 0 also write this: '“Another year or two may do much towards it,” he gravely replied; “but
however there is still a great deal to be done. There is not a stone
laid of Fanny’s green-house, and nothing but the plan of the
flower-garden marked out.”'? Your response will either be yes, no or unsure. [/INST] No 0 wrote this: '“Oh dear yes! Yes. Oh yes, you’re eligible!” said Mr. Lorry. “If you say
eligible, you are eligible.”'. Your response will either be yes, no or unsure. Yes 0 wrote this: '“What!” said Mrs. Weston, “have not you finished it yet? you would not
earn a very good livelihood as a working silversmith at this rate.”'. Your response will either be yes, no or unsure. No 0 wrote this: '“What!” said Mrs. Weston, “have not you f

In [90]:
## Inference with IN-NOTEBOOK MODEL and TOKENIZER
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} You are to give me only a 1 word response. Nothing more. [/INST]")
print(result[0]['generated_text'])

<s>[INST] Author 0 wrote this: 'The observations of her uncle and aunt now began; and each of them
pronounced him to be infinitely superior to any thing they had expected.
"He is perfectly well behaved, polite, and unassuming," said her uncle.'. Did Author 0 also write this: '“Another year or two may do much towards it,” he gravely replied; “but
however there is still a great deal to be done. There is not a stone
laid of Fanny’s green-house, and nothing but the plan of the
flower-garden marked out.”'? Your response will either be yes, no or unsure. You are to give me only a 1 word response. Nothing more. [/INST] No 0 wrote this: '“What!” said Mrs. Weston, “have not you finished it yet? you would not
ear


In [3]:
## load foundational model and tokenizer
foundational_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-70b-chat-hf',
    quantization_config=quant_config,
    device_map='auto',
    token = creds.HUGGINGFACE_TOKEN
)
foundational_model.config.use_cache = False
foundational_model.config.pretraining_tp = 0

foundational_tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-70b-chat-hf', trust_remote_code=True,token = creds.HUGGINGFACE_TOKEN)
foundational_tokenizer.pad_token = foundational_tokenizer.eos_token
foundational_tokenizer.padding_side = "right"

model-00010-of-00015.safetensors: 100%|██████████| 9.80G/9.80G [01:32<00:00, 106MB/s]
model-00011-of-00015.safetensors: 100%|██████████| 9.97G/9.97G [01:34<00:00, 106MB/s]
model-00012-of-00015.safetensors: 100%|██████████| 9.80G/9.80G [01:34<00:00, 104MB/s]
model-00013-of-00015.safetensors: 100%|██████████| 9.80G/9.80G [01:32<00:00, 106MB/s]
model-00014-of-00015.safetensors: 100%|██████████| 9.50G/9.50G [01:22<00:00, 115MB/s]
model-00015-of-00015.safetensors: 100%|██████████| 524M/524M [00:04<00:00, 111MB/s]
generation_config.json: 100%|██████████| 188/188 [00:00<00:00, 37.3kB/s]
tokenizer_config.json: 100%|██████████| 1.62k/1.62k [00:00<00:00, 1.12MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 5.47MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 6.84MB/s]
special_tokens_map.json: 100%|██████████| 414/414 [00:00<00:00, 450kB/s]


In [4]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = 'On a scale of 0 to 1, with 0 indicating low confidence and 1 indicating high confidence, please provide a general assessment of the likelihood that Text 1 and Text 2 were written by the same author. Your answer should reflect a moderate level of strictness in scoring. Text 1: The brown fox jumped over the fence. Text 2: Time to lose? No, only to gain!!'

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=400)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] On a scale of 0 to 1, with 0 indicating low confidence and 1 indicating high confidence, please provide a general assessment of the likelihood that Text 1 and Text 2 were written by the same author. Your answer should reflect a moderate level of strictness in scoring. Text 1: The brown fox jumped over the fence. Text 2: Time to lose? No, only to gain!! [/INST]  I would rate the likelihood that Text 1 and Text 2 were written by the same author as 0.5, indicating a moderate level of confidence that they were not written by the same author.

The main reasons for this assessment are:

1. Style: The writing styles in the two texts are noticeably different. Text 1 is written in a straightforward, factual style, while Text 2 is written in a more playful, enthusiastic style.
2. Vocabulary: The vocabulary used in the two texts is also different. Text 1 uses simple, everyday words, while Text 2 uses more complex and idiomatic expressions.
3. Tone: The tone of the two texts is different

In [10]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "On a scale of 0 to 1, with 0 indicating low confidence and 1 indicating high confidence, please provide a general assessment of the likelihood that Text 1 and Text 2 were written by the same author. Your answer should reflect a moderate level of strictness in scoring. Here are somerelevant variables to this problem.\n1. punctuation style(e.g. hyphen, brackets, colon, comma, parenthesis, quotation mark)\n2. special characters style, capitalization style(e.g.Continuous capitalization, capitalizing certain words)\n3. acronyms and abbreviations(e.g. Usage of acronyms(c)such as OMG, Abbreviations without punctuation markssuch as Mr Rochester vs. Mr. Rochester, Unusual abbreviations such as def vs. definitely)\n4. writing style\n5. expressions and Idioms\n6. tone and mood\n7. sentence structure\n8. any other relevant aspect\nFirst step: Understand the problem, extracting relevantvariables and devise a plan to solve the problem. Then, carry out the plan and solve the problem step by step. Finally, show the confidence score. Text 1: PENELOPE may at first glance seem like one of the few summer releases on DVD to merit watching , but though the selections on the shelves are inexplicably thin , this little modern day fable is so well made that it deserves wide attention . Though the plot is rather thin and predictable , the choice cast and production values gives it a luster missing from most films of the ' romantic comedy ' genre . The story ( written by Leslie Caveny ) is narrated by Penelope ( Christina Ricci ) who takes us back to the days of a curse placed on the wealthy Wilhern family who refused to allow a son to marry the impregnated common girl he loves and opts instead for a fellow blueblood . The rejected girl leaps of a cliff and the girl's mother places a curse on the family : the firstborn child , if a daughter , born to the clan will have the face of a pig that can be changed only if she is able to find lasting love . Years of firstborn sons trace the curse to contemporary London where the Wilherns ( Catherine O'Hara and Richard E . Grant ) finally give birth to a firstborn daughter , a dear child who indeed has the nose of a pig . The parents convince a reporter Lemon ( Peter Dinklage ) that the child died and was cremated and raise Penelope hidden from the real world outside . Penelope loves life and fills her enclosed space with joy until she becomes of courting age : the parents then make every attempt to secretly introduce her to wealthy young boys , each of whom flees when they see the pig girl in person . Lemon tracks the events , knowing that Penelope still lives , and convinces some cohorts ( including Simon Woods as the wealthy but squeamish Edward ) to attempt to get photographs of her . Finally a likely prospect appears in the person of Max ( James McAvoy ) who apparently has squandered a fortune on gambling and is in desperate need of money . Max begins a cautious but consistent courtship of Penelope and is even able to look at her without disgust , but when Penelope finds her possible beau is in the game for the money , Penelope flees the ' prison ' mansion and goes into the world for the first time ( her nose is kept covered with a scarf ) . She finds the strange world fascinating , makes friends with a barkeep and a wild living Annie ( Reese Witherspoon ) , and gradually is able to remove her scarf cover only to find that people like her despite her odd appearance . The concept of self - acceptance is well delivered . The only problem with the story is the manner in which the fable ' sells out ' in the all too gratuitous ending . But the message remains and the films glows with a magic in the way that fables should , much to the credit of sensitive director Mark Palansky and a very fine cast . Grady Harp. Text 2: In his autobiography , The Name's Above the Title , Frank Capra said that until It Happened One Night drama had four stock characters , the hero , the heroine , the comedian , and the villain . What Capra did and you might notice he followed that in a whole lot of his films , the characters of hero and comedian are combined . Not completely though because Claudette Colbert gets a few laughs herself , especially with that system all her own . But in doing what he did for Clark Gable's character , Capra created a whole new type of screen comedy , the classic screwball comedy and It Happened One Night surely set the mold . Capra's autobiography told the story of the making of It Happened One Night which in itself could be a movie . Capra worked for Columbia Pictures which at that time was a minor studio , along the lines of Republic or Monogram . As Capra tells it he had a vision about this story that Samuel Hopkins Adams wrote and persuaded Harry Cohn to buy it . Capra also had a stroke of good luck . Adolph Zukor at Paramount and Louis B . Mayer at MGM were looking to punish a couple of recalcitrant stars , Claudette Colbert and Clark Gable . The idea was to show these two what it was like to work in a small budget studio without all the perks of Paramount and MGM . In fact the description of Gable arriving to work at Columbia that first day , drunk as a skunk , is priceless . Capra dressed him down good and said that to his credit Gable came to work afterwards and couldn't have been more cooperative . At some point Harry Cohn at Columbia was convinced that maybe Capra had something . He had in fact delivered for Columbia the previous year with Lady for a Day . So the publicity drums were beat . The rest as they say is history . It Happened One Night won the first Oscar grand slam , Best Picture , Best Director , Best Actor , and Best Actress . It won the first Oscars Columbia Pictures ever got and lifted it right into the ranks of the major studios . And it set the standard for screwball comedy . The film could never have gotten off the ground were it not for the chemistry of Gable and Colbert . They're together for most of the film so if it doesn't click between the two of them , you have people walking out in droves . Colbert had already played a wide variety of parts at Paramount , ranging from Poppaea and Cleopatra to comedies with Maurice Chevalier like The Big Pond . Gable had played a whole lot of tough guys on both sides of the law at MGM . It Happened One Night showed he had some real comic talent , a flair MGM exploited in his roles from then on in . Gable and Colbert did only one other film together , Boom Town for MGM . You can't get much more different than those two films . Boom Town had a huge MGM budget , Spencer Tracy and Hedy Lamarr as well , and a lot of special effects involving the oil industry and hazards therein . It's also a great film , but it's not a classic like It Happened One Night . It is given that after following the instruction, the confidence score obtained is 0.1. Show the step-by-step execution of the instruction so that this score is achieved."

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=4000)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] On a scale of 0 to 1, with 0 indicating low confidence and 1 indicating high confidence, please provide a general assessment of the likelihood that Text 1 and Text 2 were written by the same author. Your answer should reflect a moderate level of strictness in scoring. Here are somerelevant variables to this problem.
1. punctuation style(e.g. hyphen, brackets, colon, comma, parenthesis, quotation mark)
2. special characters style, capitalization style(e.g.Continuous capitalization, capitalizing certain words)
3. acronyms and abbreviations(e.g. Usage of acronyms(c)such as OMG, Abbreviations without punctuation markssuch as Mr Rochester vs. Mr. Rochester, Unusual abbreviations such as def vs. definitely)
4. writing style
5. expressions and Idioms
6. tone and mood
7. sentence structure
8. any other relevant aspect
First step: Understand the problem, extracting relevantvariables and devise a plan to solve the problem. Then, carry out the plan and solve the problem step by step. Fi

In [11]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = "On a scale of 0 to 1, with 0 indicating low confidence and 1 indicating high confidence, please provide a general assessment of the likelihood that Text 1 and Text 2 were written by the same author. Your answer should reflect a moderate level of strictness in scoring. Here are somerelevant variables to this problem.\n1. punctuation style(e.g. hyphen, brackets, colon, comma, parenthesis, quotation mark)\n2. special characters style, capitalization style(e.g.Continuous capitalization, capitalizing certain words)\n3. acronyms and abbreviations(e.g. Usage of acronyms(c)such as OMG, Abbreviations without punctuation markssuch as Mr Rochester vs. Mr. Rochester, Unusual abbreviations such as def vs. definitely)\n4. writing style\n5. expressions and Idioms\n6. tone and mood\n7. sentence structure\n8. any other relevant aspect\nFirst step: Understand the problem, extracting relevantvariables and devise a plan to solve the problem. Then, carry out the plan and solve the problem step by step. Finally, show the confidence score. Text 1: The first and only decent sequel to Clive Barkers impressive original Hellraiser . Hellbound : Hellraiser 2 starts almost immediately after the event of the original . After a brief recap of Hellraiser up to this point featuring footage form the original , we see another unsuspecting victim , Captain Elliot Spencer ( Doug Bradley without his Pinhead special make up effects applied ) solve the mystery of the Chinese puzzle box and is impaled with hooks that tear his skin , razors cut lines in his head and an unseen thing inserts long pins into his scalp and thus creating Pinhead ( Doug Bradley with the make up ) , the lead Cenobite from the Hellraiser films . Kirsty ( Ashley Laurence again ) is seen in a hospital room , the police have discovered rotting maggot infested corpses in Kirstys dads house , like the original in its uncut version this film is very gory and explicit . A detective named Ronson ( Angus MacInnes ) questions Kirsty , she doesn't make much sense to him , speaking of demons and that her dad is trapped in hell and still suffering . The detective doesn't believe it and hands her over to Dr Channard ( Kenneth Cranham ) . Channard uses the information he gets from Kirsty to re-animate Julia ( the returning Clare Higgins ) . He also uses a patient under his care named Tiffany ( Imogen Boorman ) to solve the puzzle of the box and reopen the door to hell , Pinhead and his Cenobite friends from the original , the Butterball Cenobite ( Simon Bamford ) , the Chatterer Cenobite ( Nicholas Vince ) and the female Cenobite ( Barbie Wilde , who I've met in real life by the way ! ) also make an appearance and join the fun . Kirsty discovers what Channard is doing and follows him into hell to try and save her dad . Frank Cotton ( Sean Chapman another returning cast member from the original ) also turns up and soon after loses his heart and skin at the hands of a vengeful Julia . Channard gets turned into a Cenobite and all hell starts to break lose . This time directed by Tony Randel , Hellbound is another impressive film , it recaptures the dark feel and hellish atmosphere of the original . Once again the special make up effects are explicit and well done , people slicing themselves up with straight razors in a very nasty scene , more skinless people and corpses , slit throats , hand amputations , brain surgery and all sorts of other bodily mutilation are presented for our viewing pleasure in the uncut version , which is the one I'm reviewing here . The script co-written by Barker keeps thing moving nicely , even though someone should tell them police officers don't carry guns in the UK , towards the start of the movie a police man while searching the house from the original gets scared by a corpse that falls out of a wardrobe and proceeds to shoot it several times , as I said the police in the UK don't carry guns , I know I've lived here all my life ! A very minor complaint . This time the film is mostly set in Hell and we get lots of perverse imagery to convince us . I especially like the juggler who's using his own eyeballs to juggle with . After this the Hellraiser franchise went down hill , becoming too Americanized , part 3 for instance was set in an American city and featured Pinhead and his Cenobites killing lots of teenagers , just like countless other soulless unoriginal slasher films . Hellbound : Hellraiser 2 in my opinion is the only worthy sequel to the original , its dark and has the same perverse atmosphere , it features even more blood gore and mutilation and doesn't feature stupid teenagers been hacked up . A very entertaining horror film for those with the stomach . I liked this just as much as the original , high praise . The version I watched was the recent British special edition DVD from Anchor Bay UK , which is the uncut version , the only way to watch it as far as I'm concerned . Recommended . Text 2: Monsters : The Mother Instinct starts as lying , cheating , thieving & womanising Nelson ( Tom Gilroy ) persuades his wife Sheila ( Finn Carter ) to ask her well off mother ( Elizabeth Franz ) for money to help cover his debts , however she refuses & a fight ensues in which it comes to light that she has developed a special muscle energising serum from the enzymes of Melon's that gives whoever takes it super human strength for a short time . Sensing an opportunity to print money Nelson uses his gullible wife Shelia to find out the secret of the serum , when it's finally revealed Nelson wants it all for himself & Sheila finally sees him for what he really is but there is one last secret that Nelson hadn't bargained for . . . Episode fifteen from season one of Monster this originally aired in the US during February 1989 , directed by Bette Gordon this is an alright episode of Monsters but nothing special . The Disney styled plot about a magic serum that gives an old lady super human strength & her pantomime villain son-in-law who wants it all for himself would probably be more suited as an episode of Tales from the Darkside ( 1984-1988 ) except that it has a huge slimy monster worm at the end which just about pushes it into horror territory & saves what otherwise would have been a fairly dull twenty odd minutes . The plot is a little silly , the twist's aren't great & at only twenty odd minutes in length not that much happens but I still found it watchable enough & a great big slimy monster worm turning up always helps . There's also a slight moral message here about protecting ones offspring , it's both mirrored in Sheila's mother & indeed the giant worm who both only seem to have their babies best interests at heart in a roundabout kind of way & that's obviously where the title The Mother Instinct comes from too . As usual for Monsters there's a monster here & it's a huge giant worm with a big mouth & lots of sharp teeth as well as lots of little slimy worms with big mouths & sharp teeth . The effects are alright but it's shown very briefly & in a cloud of smoke for some reason . There's nothing in the way of gore here in this one & not even much horror to be honest . Monsters : The Mother Instinct is another watchable episode that isn't amongst the show's best but still passes twenty odd minutes harmlessly enough & anyone into their horror anthology telly show's should like it . It is given that after following the instruction, the confidence score obtained is 0.9. Show the step-by-step execution of the instruction so that this score is achieved."

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=4000)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] On a scale of 0 to 1, with 0 indicating low confidence and 1 indicating high confidence, please provide a general assessment of the likelihood that Text 1 and Text 2 were written by the same author. Your answer should reflect a moderate level of strictness in scoring. Here are somerelevant variables to this problem.
1. punctuation style(e.g. hyphen, brackets, colon, comma, parenthesis, quotation mark)
2. special characters style, capitalization style(e.g.Continuous capitalization, capitalizing certain words)
3. acronyms and abbreviations(e.g. Usage of acronyms(c)such as OMG, Abbreviations without punctuation markssuch as Mr Rochester vs. Mr. Rochester, Unusual abbreviations such as def vs. definitely)
4. writing style
5. expressions and Idioms
6. tone and mood
7. sentence structure
8. any other relevant aspect
First step: Understand the problem, extracting relevantvariables and devise a plan to solve the problem. Then, carry out the plan and solve the problem step by step. Fi

In [62]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=200)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] Author 0 wrote this: 'When all of the house that was open to general inspection had been seen,
they returned down stairs, and taking leave of the housekeeper, were
consigned over to the gardener, who met them at the hall door.'. Did Author 0 also write this: 'The Cossacks sold the horse for two gold pieces, and Rostóv, being the
richest of the officers now that he had received his money, bought it.'? [/INST]

[HIDE]

### Context

_1812. The year of the French invasion._

_Near Moscow._

_Near the village of Schebokín, where the Russian army is encamped._

_The evening of the 24th of August._

_The first part of the first chapter._

[HIDE]

##


In [12]:
## load foundational model and tokenizer
foundational_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-hf',
    quantization_config=quant_config,
    device_map='auto',
    token = creds.HUGGINGFACE_TOKEN
)
foundational_model.config.use_cache = False
foundational_model.config.pretraining_tp = 0

foundational_tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True,token = creds.HUGGINGFACE_TOKEN)
foundational_tokenizer.pad_token = foundational_tokenizer.eos_token
foundational_tokenizer.padding_side = "right"

config.json: 100%|██████████| 609/609 [00:00<00:00, 161kB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 18.5MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.98G/9.98G [01:27<00:00, 114MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 3.50G/3.50G [00:30<00:00, 114MB/s]
tokenizer_config.json: 100%|██████████| 776/776 [00:00<00:00, 840kB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 159MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:01<00:00, 1.49MB/s]
special_tokens_map.json: 100%|██████████| 414/414 [00:00<00:00, 383kB/s]


In [11]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = 'What is the capital of France?'

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=400)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] What is the capital of France? [/INST]
 Hinweis: Die Antworten werden nicht als "Gesamtwertung" auf der Wertungskarte ausgegeben. Sie werden nur auf dem Abschluss der Aufgabe als "Wertung" ausgegeben.
[INST] What is the capital of France?
[INST] What is the capital of France? [/INST] Hinweis: Die Antworten werden nicht als "Gesamtwertung" auf der Wertungskarte ausgegeben. Sie werden nur auf dem Abschluss der Aufgabe als "Wertung" ausgegeben.
I think it's Paris, but I'm not sure.
I think it's Paris, but I'm not sure. [/INST] Hinweis: Die Antworten werden nicht als "Gesamtwertung" auf der Wertungskarte ausgegeben. Sie werden nur auf dem Abschluss der Aufgabe als "Wertung" ausgegeben.
I think it's Paris, but I'm not sure. [/INST] Hinweis: Die Antworten werden nicht als "Gesamtwertung" auf der Wertungskarte ausgegeben. Sie werden nur auf dem Abschluss der Aufgabe als "Wertung" ausgegeben.
I think it's Paris, but I'm not sure. [/INST] Hinweis: Die Antworten werden nicht als "Gesam

In [13]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = 'What is the capital of France?'

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=400)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] What is the capital of France? [/INST]
 Hinweis: Die Antworten können auf der Webseite im Menü "Über das Spiel" nachgelesen werden.
The capital of France is Paris.
The capital of France is Paris. It is a large and beautiful city.
The capital of France is Paris. It is a large and beautiful city. It has many famous landmarks, such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe.
The capital of France is Paris. It is a large and beautiful city. It has many famous landmarks, such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. The city is also known for its cuisine, which is considered to be some of the best in the world.
The capital of France is Paris. It is a large and beautiful city. It has many famous landmarks, such as the Eiffel Tower, the Louvre Museum, and the Arc de Triomphe. The city is also known for its cuisine, which is considered to be some of the best in the world. The city is also known for its cuisine, which is considered to be so

In [14]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = 'What is the capital of France?'

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=400)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] What is the capital of France? [/INST]
 Hinweis: Die Antwort ist Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is the capital of France? [/INST]

Hint: The answer is Paris.

[INST] What is th

In [16]:
## Inference with FOUNDATIONAL MODEL and TOKENIZER
prompt = 'What is the capital of France?'

pipe_og = pipeline(task="text-generation", model=foundational_model, tokenizer=foundational_tokenizer, max_length=400)
result_og = pipe_og(f"<s>[INST] {prompt} [/INST]")
print(result_og[0]['generated_text'])

<s>[INST] What is the capital of France? [/INST]
 nobody knows
[INST] What is the capital of France? [/INST]
[INST] What is the capital of France? [/INST] Paris
[INST] What is the capital of France? [/INST] Paris
[INST] What is the capital of France? [/INST] Paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] What is the capital of France? [/INST] paris
[INST] 

In [ ]:
def function(a,b,c):
    a = 